<a href="https://colab.research.google.com/github/WooHyunJ/Final-Team9/blob/main/ttttest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/TMDB_with_directors.csv")

In [ ]:
# 제작비용 10000달러 이상
df = df[(df['budget'] > 10000) & (df['revenue'] > 0)]
#ROI 계산
df['ROI'] = ((df['revenue'] - df['budget']) / df['budget']) * 100

/tmp/ipython-input-2565592165.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ROI'] = ((df['revenue'] - df['budget']) / df['budget']) * 100


In [ ]:
conditions = [
    (df['revenue'] / df['budget']) >= 3,
    (df['revenue'] / df['budget']) >= 2.5,
    (df['revenue'] / df['budget']) >= 2,
]
choices = [3, 2, 1]
df['SR'] = np.select(conditions, choices, default=0)

/tmp/ipython-input-1957793358.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SR'] = np.select(conditions, choices, default=0)


In [ ]:
# --- 1. 데이터 준비 (결측치 처리) ---
# keywords 칼럼의 빈 값(NaN)을 에러 방지를 위해 빈 문자열로 채웁니다.
df['keywords'] = df['keywords'].fillna('')

# --- 2. `explode`로 행 확장 ---
# 쉼표(,)를 기준으로 분리하고 explode로 행을 확장합니다.
df['keywords'] = df['keywords'].str.split(',')
exploded_df = df.explode('keywords')


# --- 3. 키워드 정제 (공백 제거) ---
# 각 키워드의 앞뒤 공백을 제거하고, 빈 문자열 행을 제거합니다.
exploded_df['keywords'] = exploded_df['keywords'].str.strip()
exploded_df = exploded_df[exploded_df['keywords'] != '']


# --- 4. 키워드 빈도수 계산 ---
# 전체 확장된 데이터에서 키워드별 빈도수를 계산합니다.
keyword_counts = exploded_df['keywords'].value_counts()


# --- 5. 빈도수 5 이상인 키워드 '목록' 생성 ---
# 빈도수가 5 이상인 키워드만 골라냅니다.
frequent_keywords = keyword_counts[keyword_counts >= 5]
# 필터링에 사용할 수 있도록 키워드 이름만 리스트로 추출합니다.
frequent_keyword_list = frequent_keywords.index.tolist()


# --- 6. 최종 필터링 ---
# 'exploded_df'에서 'keywords' 칼럼의 값이 'frequent_keyword_list'에 포함된 행만 남깁니다.
final_df = exploded_df[exploded_df['keywords'].isin(frequent_keyword_list)].copy()


/tmp/ipython-input-1123285324.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].fillna('')
/tmp/ipython-input-1123285324.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].str.split(',')


In [ ]:
columns_to_process = ['spoken_languages', 'production_countries', 'genres', 'production_companies']
processed_df = final_df.copy()

In [ ]:
# 각 칼럼에 대해 반복적으로 작업 수행
for col in columns_to_process:
    # 1. 쉼표(,)를 기준으로 문자열을 분할하여 리스트로 만듭니다.
    #    .str 접근자는 NaN 값을 알아서 처리해 주므로 편리합니다.
    processed_df[col] = processed_df[col].str.split(',')

    # 2. 리스트의 각 항목을 별도의 행으로 분리(explode)합니다.
    processed_df = processed_df.explode(col)

    # 3. 분리된 각 값의 양쪽 공백을 제거(strip)합니다.
    processed_df[col] = processed_df[col].str.strip()

# (선택 사항) 공백 제거 후 빈 문자열이 생겼다면 NaN으로 바꾸고 제거
processed_df[columns_to_process] = processed_df[columns_to_process].replace('', np.nan)
processed_df.dropna(subset=columns_to_process, inplace=True)

# 최종 결과 확인
print(processed_df.head())

      id      title  vote_average  vote_count    status release_date  \
0  27205  Inception         8.364       34495  Released   2010-07-15   
0  27205  Inception         8.364       34495  Released   2010-07-15   
0  27205  Inception         8.364       34495  Released   2010-07-15   
0  27205  Inception         8.364       34495  Released   2010-07-15   
0  27205  Inception         8.364       34495  Released   2010-07-15   

     revenue  runtime  adult                     backdrop_path  ...  \
0  825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   
0  825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   
0  825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   
0  825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   
0  825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   

                        poster_path                               tagline  \
0  /oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg  Your mind is the scene of 

In [ ]:
processed_df = processed_df.dropna(subset = ['production_countries','production_companies','spoken_languages','release_date', 'director','genres'])

In [ ]:
processed_df

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords,director,ROI,SR
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,Action,Legendary Pictures,United Kingdom,English,rescue,크리스토퍼 놀란,415.957978,3
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,Action,Syncopy,United Kingdom,English,rescue,크리스토퍼 놀란,415.957978,3
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,Action,Warner Bros. Pictures,United Kingdom,English,rescue,크리스토퍼 놀란,415.957978,3
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,Science Fiction,Legendary Pictures,United Kingdom,English,rescue,크리스토퍼 놀란,415.957978,3
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,Science Fiction,Syncopy,United Kingdom,English,rescue,크리스토퍼 놀란,415.957978,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15887,822563,Priyappettavar,0.000,0,Released,2021-04-22,69000,124,False,NaN,...,/9ZPdfxlgYhJrHBpYLmWsRp9aykG.jpg,A movie for the patriots,War,Bharatha Kshethra,India,Malayalam,tree,Khader Moidu,-83.571429,0
15887,822563,Priyappettavar,0.000,0,Released,2021-04-22,69000,124,False,NaN,...,/9ZPdfxlgYhJrHBpYLmWsRp9aykG.jpg,A movie for the patriots,Drama,Bharatha Kshethra,India,English,patriotism,Khader Moidu,-83.571429,0
15887,822563,Priyappettavar,0.000,0,Released,2021-04-22,69000,124,False,NaN,...,/9ZPdfxlgYhJrHBpYLmWsRp9aykG.jpg,A movie for the patriots,War,Bharatha Kshethra,India,English,patriotism,Khader Moidu,-83.571429,0
15887,822563,Priyappettavar,0.000,0,Released,2021-04-22,69000,124,False,NaN,...,/9ZPdfxlgYhJrHBpYLmWsRp9aykG.jpg,A movie for the patriots,Drama,Bharatha Kshethra,India,Malayalam,patriotism,Khader Moidu,-83.571429,0


In [ ]:
processed_df.to_csv("___data.csv", index=False, encoding='utf-8-sig')